In [44]:
import pandas as pd

In [45]:
#a)
def add_key(dict, key):
    if key not in dict:
        dict[key] = {
            "in": [],
            "out": []
        }
    return dict

def fill_dict(raw_data):
    graph = {}
    for from_node, to_node in zip(raw_data[" FromNodeId"], raw_data["ToNodeId"]):
        graph = add_key(graph, from_node)
        graph = add_key(graph, to_node)
        graph[from_node]["out"].append(to_node)
        graph[to_node]["in"].append(from_node)
    return graph

In [46]:
path = "data/stanford_small.txt"
raw_data = pd.read_csv(path, delimiter="\t", header=0, comment="#")
print(raw_data.head(10))
graph = fill_dict(raw_data)
print(len(graph))
for i, (k, v) in enumerate(graph.items()):
    print(k, v)
    if i == 9:
        break

    FromNodeId  ToNodeId
0            1      6548
1            1     15409
2         6548     57031
3        15409     13102
4            2     17794
5            2     25202
6            2     53625
7            2     54582
8            2     64930
9            2     73764
1587
1 {'in': [], 'out': [6548, 15409]}
6548 {'in': [1], 'out': [57031]}
15409 {'in': [1], 'out': [13102]}
57031 {'in': [6548], 'out': []}
13102 {'in': [15409], 'out': []}
2 {'in': [252915, 246897, 251658, 280935, 213966, 243294, 225119, 241596, 178642, 210870, 204189, 190453, 204604, 181714, 164599, 175799, 163550, 140864, 115926, 73764, 105730, 98628, 84477, 105318, 100193, 102355, 64930, 54582, 53625, 25202, 17794], 'out': [17794, 25202, 53625, 54582, 64930, 73764, 84477, 98628, 100193, 102355, 105318, 105730, 115926, 140864, 163550, 164599, 175799, 178642, 181714, 190453, 204189, 204604, 210870, 213966, 225119, 241596, 243294, 246897, 251658, 252915, 280935]}
17794 {'in': [2], 'out': [2]}
25202 {'in': [2], 'out'

In [48]:
def pagerank_distributed(graph, iterations, damping_factor):
    N = len(graph)
    rank = 1 / N
    rdm_tele_val = (1 - damping_factor) / N
    graph = {node: {**data, 'rank_old': rank} for node, data in graph.items()}
    for t in range(iterations):
        graph = {node: {**data, 'rank_new': graph[node]["rank_old"] * rdm_tele_val} for node, data in graph.items()}
        for node in graph:
            old_rank = graph[node]['rank_old']
            out_nodes = graph[node]['out']
            out_degree = len(out_nodes)
            for out_node in out_nodes:
                graph[out_node]['rank_new'] += damping_factor * old_rank / out_degree
    return graph

In [52]:
graphs_w_pagerank = pagerank_distributed(graph, 5, 0.8)
sum_values = 0
for node, data in graphs_w_pagerank.items():
    #print(node, data["rank_new"])
    sum_values += data["rank_new"]
print(sum_values)



0.09993698802772472
